In [1]:
''' Laufzeit grob 1.4h 
tfidf: Average Precision@10: 0.379369798
Average Recall@10: 0.001416642
word2cvec: Average Precision@10: 0.385087198
Average Recall@10: 0.001694450
bert: Average Precision@10: 0.419193421
Average Recall@10: 0.002046413
blf: Average Precision@10: 0.409848410
Average Recall@10: 0.001744662
ivec: Average Precision@10: 0.434363853
Average Recall@10: 0.001946139
mfcc stats: Average Precision@10: 0.428874356
Average Recall@10: 0.001762932
music_nn: Average Precision@10: 0.481252477
Average Recall@10: 0.002228871
[(0.3793697978596908, 0.0014166415191423345),
 (0.38508719778042017, 0.0016944503811388812),
 (0.41919342053111375, 0.002046413141468218),
 (0.4098484097889626, 0.0017446617681295038),
 (0.4343638525564804, 0.001946138801676427),
 (0.4288743559254855, 0.0017629319493361406),
 (0.48125247720967107, 0.0022288708476247863)]
with lda
tfidf (0.3468985334918747, 0.0015012111002040932)
with pca
word2vec (0.33546373365041615, 0.0010239388128254544)
UMAP(n_neighbors=15, min_dist=0.1, n_components=50)
(0.3318370986920333, 0.0010228425352478046)
'''

' Laufzeit grob 1.4h \ntfidf: Average Precision@10: 0.379369798\nAverage Recall@10: 0.001416642\nword2cvec: Average Precision@10: 0.385087198\nAverage Recall@10: 0.001694450\nbert: Average Precision@10: 0.419193421\nAverage Recall@10: 0.002046413\nblf: Average Precision@10: 0.409848410\nAverage Recall@10: 0.001744662\nivec: Average Precision@10: 0.434363853\nAverage Recall@10: 0.001946139\nmfcc stats: Average Precision@10: 0.428874356\nAverage Recall@10: 0.001762932\nmusic_nn: Average Precision@10: 0.481252477\nAverage Recall@10: 0.002228871\n[(0.3793697978596908, 0.0014166415191423345),\n (0.38508719778042017, 0.0016944503811388812),\n (0.41919342053111375, 0.002046413141468218),\n (0.4098484097889626, 0.0017446617681295038),\n (0.4343638525564804, 0.001946138801676427),\n (0.4288743559254855, 0.0017629319493361406),\n (0.48125247720967107, 0.0022288708476247863)]\nwith lda\ntfidf (0.3468985334918747, 0.0015012111002040932)\nwith pca\nword2vec (0.33546373365041615, 0.00102393881282545

# Task 2 : Extend your framework with audio-based retrieval systems and with evaluation metrics

### Team E

**Don't forget to update the version number after making changes** 

Version 2.0.2
Date: 11/12/2023

## Import Libraries  

In [2]:
import numpy as np
import pandas as pd
import json 
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA
import umap

In [3]:
# import utility functions 
from ret import read_data # utility func to load data
from ret import get_id_from_info # utility func to return id by entering song's info
from ret import display_res # utility func to display results 
from ret import get_genre #utility func to get the genres from a list of id´s
from ret import get_genre_from_query #utility func to get the id and genre from the query
from ret import get_genre_from_ids #utility func to get the id and genre from the retrieved results

# import the retrieval systems 
from ret import random_baseline # baseline retrieval system that returns random results 
from ret import text_based # modularized text based retrieval system
from ret import audio_based # modularized audio based retrieval system

# import wrapper function to calculate cosine similarity
from ret import cos_sim 

# import evaluation functions 
from ret import gen_cov_10
from ret import ndcg_score
from ret import gen_div_10
from ret import calculate_precision_at_k
from ret import calculate_recall_at_k_vectorized
from ret import plot_precision_recall_curve
from ret import get_avg_recall_at_k
from ret import get_avg_precision_at_k

In [4]:
#!pip install sklearn

In [5]:
#!pip install umap-learn

## Load Data

In [6]:
"""
Please put data files in ""./data/" before use 
"""

info = read_data("information")
genres = read_data("genres")

# text embeddings 
tfidf = read_data("lyrics_tf-idf")
word2vec = read_data("lyrics_word2vec")
bert = read_data("lyrics_bert")

# audio embeddings
blf_correlation = read_data("blf_correlation")
ivec256 = read_data("ivec256")
mfcc_stats = read_data("mfcc_stats")
musicnn = read_data("musicnn")

In [7]:
def convert2matrix(embedding):
    embedding = embedding.sort_values(by='id')
    # Konvertieren Sie den DataFrame in ein NumPy-ndarray
    embedding_array = embedding.iloc[:, 2:].to_numpy()
    # Konvertieren Sie das NumPy-ndarray in eine NumPy-Matrix
    # initialize the LDA model with the desired parameters
    #lda = LatentDirichletAllocation(n_components=50, max_iter=5, learning_method='online', learning_offset=50., random_state=0)

    '''# Apply UMAP
    umap_model = umap.UMAP(n_neighbors=10, min_dist=0.2, n_components=50)  # Adjust parameters as needed
    print("Starting UMAP")
    embedding_umap = umap_model.fit_transform(embedding_array)
    print("Stopping UMAP")'''
    
    # Convert the UMAP result to a matrix
    embedding_matrix = np.matrix(embedding_array)
    
    # Transponieren Sie die Matrix mit der transpose() Methode
    #embedding_transpose = embedding_matrix.transpose()
    #embedding_mm_fused = data_fusion(tfidf[:10094], musicnn)
    # Transponieren Sie die Matrix mit der transpose() Methode
    embedding_transpose = embedding_matrix.transpose()
    return embedding_matrix,embedding_transpose

In [8]:
def data_fusion(embedding1, embedding2):
    matrix1, matrix1b = convert2matrix(embedding1)
    matrix2, matrix2b = convert2matrix(embedding2)

    # Ensure the matrices have the same number of rows
    if matrix1.shape[0] != matrix2.shape[0]:
        raise ValueError("Matrices must have the same number of rows for concatenation.")

    # Horizontally stack the matrices
    fused_matrix = np.hstack((matrix1, matrix2))

    return fused_matrix

In [9]:
def make2embeddings1(embedding1, embedding2):
    embedding_array = data_fusion(embedding1.head(10094),embedding2 )
    #Apply UMAP
    # Convert to NumPy array
    embedding_array = np.asarray(embedding_array)
    umap_model = umap.UMAP(n_neighbors=10, min_dist=0.2, n_components=50)  # Adjust parameters as needed
    print("Starting UMAP")
    embedding_umap = umap_model.fit_transform(embedding_array)
    print("Stopping UMAP")
    # Convert the UMAP result to a matrix
    embedding_matrix = np.matrix(embedding_umap)
    return embedding_matrix, embedding_matrix.transpose()

In [10]:
# Convert the UMAP result to a matrix
#embedding_matrix = np.matrix(embedding_umap)
    
#embedding_matrix.shape

In [11]:
#embedding_matrix

In [12]:
#lda.shape

In [13]:
musicnn

,id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,01rMxQv6vhyE1oQX,0.251818,0.004347,0.078437,0.056584,0.012362,0.088567,0.061890,0.171026,0.045924,...,0.055074,0.011236,0.000931,0.001090,0.007918,0.020941,0.060759,0.003712,0.040641,0.001380
1,01Yfj2T3YTwJ1Yfy,0.129031,0.001123,0.006577,0.082643,0.002042,0.095536,0.049312,0.686902,0.154331,...,0.004771,0.005219,0.000255,0.000172,0.008303,0.026122,0.002091,0.243107,0.005494,0.000828
2,01gyRHLquwXDlhkO,0.026824,0.000873,0.009361,0.313308,0.002543,0.126715,0.176702,0.053236,0.131477,...,0.024470,0.036648,0.000088,0.000145,0.006926,0.038333,0.005486,0.003181,0.020700,0.001349
3,02RGE9FNH65RtMS7,0.000452,0.001396,0.002990,0.880924,0.000339,0.066310,0.491459,0.002746,0.605974,...,0.000146,0.087928,0.000015,0.000003,0.030459,0.012542,0.000006,0.000068,0.000064,0.000004
4,02ZnlCGZEbkfCDxo,0.219504,0.011687,0.054232,0.060582,0.037868,0.158267,0.081095,0.156428,0.076932,...,0.097831,0.015530,0.002125,0.001461,0.013634,0.020431,0.035571,0.004393,0.062585,0.002477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,zzgS4ZqyswamEWNj,0.133540,0.000634,0.015991,0.112173,0.002589,0.082022,0.077761,0.248482,0.115401,...,0.193905,0.011680,0.000121,0.000191,0.005070,0.013650,0.023206,0.003180,0.134805,0.000809
10090,zzoFYDMlqU1X2zz1,0.353242,0.210615,0.211620,0.011774,0.230070,0.025087,0.027354,0.025995,0.042540,...,0.026651,0.001608,0.019035,0.017204,0.022030,0.010009,0.044221,0.001399,0.018508,0.002537
10091,zzpkRCGA5ud8q4mv,0.311070,0.030005,0.053963,0.041861,0.076557,0.146726,0.050487,0.154200,0.102354,...,0.031082,0.013495,0.004440,0.004140,0.015955,0.019282,0.096946,0.007830,0.021242,0.001236
10092,ZzVzT19g8bLDuQX4,0.036959,0.002318,0.020133,0.407093,0.005032,0.105486,0.262142,0.038931,0.139739,...,0.035586,0.055873,0.000230,0.000309,0.012591,0.049588,0.007273,0.002378,0.028774,0.004017


In [14]:
#embedding_mm_fused = data_fusion(tfidf[:10094], musicnn)

In [15]:
 genres.drop(7298, axis=0, inplace=True)

In [16]:
def generate_cos_sim_matrix(embedding1, embedding2):
    print(embedding1.shape)
    print(embedding2.shape)
    #embedding1.drop(7298, axis=0, inplace=True)
    #embedding2.drop(7298, axis=0, inplace=True)
    embedding_matrix,embedding_transpose = make2embeddings1(embedding1, embedding2)
    #embedding_matrix,embedding_transpose = convert2matrix(embedding)


    # Überprüfen Sie die Form der transponierten Matrizen
    print(embedding_matrix.shape) # (1328, 10093)
    #print(len(blf_correlation), len(blf_correlation_transpose_zip[0])) # (1328, 10093)
    # Berechnen Sie die Kosinusähnlichkeitsmatrix
    cos_sim_matrix = np.triu(embedding_matrix.dot(embedding_transpose) / (np.linalg.norm(embedding_matrix, axis=1) * np.linalg.norm(embedding_matrix, axis=1)[:, None]))
    # Fill the lower triangular part with the same values as the upper triangular part
    cos_sim_matrix = cos_sim_matrix + cos_sim_matrix.T - np.diag(cos_sim_matrix.diagonal())

    # Create a dataframe from the numpy matrix
    df = pd.DataFrame(cos_sim_matrix)

    # Assign the ids to the index and columns
    df.index = embedding1.iloc[:, 0]
    #df.index = df.index.str.strip('(),')
    df.columns = embedding1.iloc[:, 0]
    #df.columns = df.columns.str.strip('(),')

    return df

In [17]:
def change_to_set(s):
    # Wandeln Sie den String in eine Liste von Genres um
    l = s.strip("[]").split(",")
    # Entfernen Sie die Anführungszeichen um jedes Genre
    l = [g.strip("'") for g in l]
    # Wandeln Sie die Liste in ein Set um
    s = set(l)
    return s


In [18]:
def change_list_to_sets(list):
    array = []
    # Iterieren Sie über die Spalte in Ihrem DataF
    ''' probieren für mehr performance def change_list_to_sets(list):
    # Wandeln Sie die Liste von Strings in eine Liste von Sets um
    return [change_to_set(s) for s in list]
    '''
    for s in list:
        s =change_to_set(s)
        # Fügen Sie das Set zum Array hinzu
        array.append(s)
    return array

In [19]:
#df = pd.merge(df, genres, on='id')
# Erstellen Sie ein leeres Array aus Sets
def get_relevancy_count(genre_query,genre_other_songs):
    array = change_list_to_sets(genre_other_songs)

    #print("Das Array aus Sets ist fertig")
    #print(array)



    # Initialize a counter
    count = 0

    print("Loop through the list of sets")
    '''Try for better performance# Erstellen Sie eine neue Liste mit 1 für jedes Element, das die Bedingung erfüllt
    newlist = [1 for s in array if genre_query.intersection(s)]

    # Berechnen Sie die Summe der neuen Liste, um den Zähler zu erhalten
    count = sum(newlist)
    '''
    for s in array:
        # Check if the intersection is not empty
        if genre_query.intersection(s):#
        # Increment the counter
            count += 1

    print("done")
    #print(count) # 1
    return count

In [20]:
def calc_avg_recall_precision_at_10(df):
    # Define the value of k
    k = 10

    i = 1


    # Define an empty list to store the precision and recall scores
    precision_scores = []
    recall_scores = []

    # Loop over each query track
    for query in df.id:
        print(i)
        i = i+1
        print(query)
        #print("Sort the other tracks by their similarity scores in descending order and get genres")
        ranked_list = df.sort_values(by=query, ascending=False).genre.to_list()

        #print("Select the top k tracks from the ranked list")
        retrieved_list = ranked_list[1:k+1]
        #print(retrieved_list)

        # Get the genre of the query track
        query_genre = genres[genres['id'] == query]['genre'].iloc[0]
        #print(query_genre)
        query_genre = change_to_set(query_genre)
        #print(query_genre)

        #get genres  as set of top k tracks
        top_k_genre = change_list_to_sets(retrieved_list)
        #print(top_k_genre)
        #print("get count of all relevant tracks in dataset")
        all_relevant = get_relevancy_count(query_genre,df['genre'])
        #print("get count of all relevant tracks in top k")
        k_relevant = get_relevancy_count(query_genre,retrieved_list)

        recall =  k_relevant / all_relevant
        #print(recall)

        precision = k_relevant/k
        #print(precision)




        # Append the precision and recall scores to the list
        precision_scores.append(precision)
        recall_scores.append(recall)

    # Calculate the average precision and recall over all query tracks
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)

    # Print the results
    print(f'Average Precision@{k}: {avg_precision:.9f}')
    print(f'Average Recall@{k}: {avg_recall:.9f}')
    
    return avg_precision, avg_recall


In [21]:
def get_avg_precision_recall_at_10(embedding1, embedding2):
    print("generate matrix")
    
    cos_sim_matrix_df = generate_cos_sim_matrix(embedding1, embedding2)
    print("generate matrix done")
    cos_sim_matrix_df = pd.merge(cos_sim_matrix_df, genres, on='id')
    avg_precision, avg_recall = calc_avg_recall_precision_at_10(cos_sim_matrix_df)
    return avg_precision, avg_recall

In [ ]:
get_avg_precision_recall_at_10(tfidf.head(10094), musicnn)

generate matrix
(10094, 1001)
(10094, 51)
Starting UMAP
Stopping UMAP
(50, 10094)
generate matrix done
1
h48f46ZsT9h0Z5Dm
Loop through the list of sets
done
Loop through the list of sets
done
2
PV5EXN6AIVBqvsLO
Loop through the list of sets
done
Loop through the list of sets
done
3
z0PzOinYUixzCTGQ
Loop through the list of sets
done
Loop through the list of sets
done
4
LZUFTnAB77X15RSz
Loop through the list of sets
done
Loop through the list of sets
done
5
nASk24WIR6Eihbut
Loop through the list of sets
done
Loop through the list of sets
done
6
eFEY5JiDF3ZLpXBZ
Loop through the list of sets
done
Loop through the list of sets
done
7
dH8L26zKHiANebSj
Loop through the list of sets
done
Loop through the list of sets
done
8
NTcf5L6mTv4vowN1
Loop through the list of sets
done
Loop through the list of sets
done
9
VAWiymoCIYxhae3J
Loop through the list of sets
done
Loop through the list of sets
done
10
2H91WLAd7ZZJvAiw
Loop through the list of sets
done
Loop through the list of sets
done
11
Ycd

Loop through the list of sets
done
Loop through the list of sets
done
93
lK1wd2EwujDcpRFG
Loop through the list of sets
done
Loop through the list of sets
done
94
tAoMaVf7KD5NH2es
Loop through the list of sets
done
Loop through the list of sets
done
95
mWPBuaxvIxOTJ6FQ
Loop through the list of sets
done
Loop through the list of sets
done
96
zI9DjkxOs5x1EhJQ
Loop through the list of sets
done
Loop through the list of sets
done
97
K6RKLKDPO6CUwwAq
Loop through the list of sets
done
Loop through the list of sets
done
98
9SUQfSsDIBOh3QSV
Loop through the list of sets
done
Loop through the list of sets
done
99
gLzftruaNc9LAp5A
Loop through the list of sets
done
Loop through the list of sets
done
100
JT4j4AEG4zHM5Fjv
Loop through the list of sets
done
Loop through the list of sets
done
101
JRrgrwMPTMsaJviI
Loop through the list of sets
done
Loop through the list of sets
done
102
M0eAVOPXuqVSntZ7
Loop through the list of sets
done
Loop through the list of sets
done
103
4tJX2tH6XLHLmUn9
Loop t

done
Loop through the list of sets
done
183
uvRBt3gxIumKNPnQ
Loop through the list of sets
done
Loop through the list of sets
done
184
rmPupjl0UzjnWAly
Loop through the list of sets
done
Loop through the list of sets
done
185
MT7QwxsMtAHhfL1l
Loop through the list of sets
done
Loop through the list of sets
done
186
vtPQZhVvjfJP0Z5n
Loop through the list of sets
done
Loop through the list of sets
done
187
lOyErwrvnenJb2TZ
Loop through the list of sets
done
Loop through the list of sets
done
188
UqvbMKf8wsQe9o2X
Loop through the list of sets
done
Loop through the list of sets
done
189
5BOpMCptzt97VnFI
Loop through the list of sets
done
Loop through the list of sets
done
190
5GeGSRKmYK8ny8Gi
Loop through the list of sets
done
Loop through the list of sets
done
191
qhbMmz339DsvlwXc
Loop through the list of sets
done
Loop through the list of sets
done
192
e0xraBYvAOVP3hlY
Loop through the list of sets
done
Loop through the list of sets
done
193
dLyOVPsfSra2qtzz
Loop through the list of sets

Loop through the list of sets
done
274
CT7e8gmciDXA0wV7
Loop through the list of sets
done
Loop through the list of sets
done
275
GsoigemhOI9q2FmI
Loop through the list of sets
done
Loop through the list of sets
done
276
KngZ0mHlw5CvVmqC
Loop through the list of sets
done
Loop through the list of sets
done
277
mKyhP8m8am4hnodk
Loop through the list of sets
done
Loop through the list of sets
done
278
V1W5M19l8qnL7MYW
Loop through the list of sets
done
Loop through the list of sets
done
279
8MGRZuoU8JUBwPVt
Loop through the list of sets
done
Loop through the list of sets
done
280
k7Nh93qJae1HAoRo
Loop through the list of sets
done
Loop through the list of sets
done
281
6tMKqIJL1BFcRqSh
Loop through the list of sets
done
Loop through the list of sets
done
282
59uRghNSX1RUlUrc
Loop through the list of sets
done
Loop through the list of sets
done
283
1VtWYqp59eTfRKXY
Loop through the list of sets
done
Loop through the list of sets
done
284
mY2ljTlP7tbg9FP1
Loop through the list of sets
done

Loop through the list of sets
done
Loop through the list of sets
done
366
uyiAoox9wCrrVoQW
Loop through the list of sets
done
Loop through the list of sets
done
367
qQQRjhfK2pGy0mit
Loop through the list of sets
done
Loop through the list of sets
done
368
IRADX2ZnCgKJlryG
Loop through the list of sets
done
Loop through the list of sets
done
369
iGBVp7u4dKNl0y7U
Loop through the list of sets
done
Loop through the list of sets
done
370
TOTiGiL4qATx6qyl
Loop through the list of sets
done
Loop through the list of sets
done
371
ePckwlmr1ngZuCOp
Loop through the list of sets
done
Loop through the list of sets
done
372
mztrw5nXNL9Qwo8O
Loop through the list of sets
done
Loop through the list of sets
done
373
eroCtRu0ruQbP71L
Loop through the list of sets
done
Loop through the list of sets
done
374
MYqWTX40nU4n00r3
Loop through the list of sets
done
Loop through the list of sets
done
375
51XYIfDaAUgQXb5u
Loop through the list of sets
done
Loop through the list of sets
done
376
llfiyRXxPeAoahRS

Loop through the list of sets
done
Loop through the list of sets
done
457
FLCk8Zl1mYItuahn
Loop through the list of sets
done
Loop through the list of sets
done
458
P9eiLOQ74QZTOpi5
Loop through the list of sets
done
Loop through the list of sets
done
459
XCYQhEAVQVN6yVfG
Loop through the list of sets
done
Loop through the list of sets
done
460
O1YKKFA84K1x5wFn
Loop through the list of sets
done
Loop through the list of sets
done
461
BRj9LADMeIhnsrp3
Loop through the list of sets
done
Loop through the list of sets
done
462
qPR0RhH84VAcLrN4
Loop through the list of sets
done
Loop through the list of sets
done
463
Y26z1h76HAfA2oca
Loop through the list of sets
done
Loop through the list of sets
done
464
Utj915yMnJa0UvyX
Loop through the list of sets
done
Loop through the list of sets
done
465
Cur2zIG2wkc8LSLV
Loop through the list of sets
done
Loop through the list of sets
done
466
No8luSxFWQj9cQi1
Loop through the list of sets
done
Loop through the list of sets
done
467
9JoD1DcnmEVCA2i8

Loop through the list of sets
done
Loop through the list of sets
done
549
F0OeKHcvWROLs1Qm
Loop through the list of sets
done
Loop through the list of sets
done
550
p05pB6stI6vNPsv3
Loop through the list of sets
done
Loop through the list of sets
done
551
WWvFA8zXcdjn4mZB
Loop through the list of sets
done
Loop through the list of sets
done
552
czcUOlUcJQjgLWwO
Loop through the list of sets
done
Loop through the list of sets
done
553
rJMmLMMyVqoVT9fL
Loop through the list of sets
done
Loop through the list of sets
done
554
H5NawoGCVfzI8rKs
Loop through the list of sets
done
Loop through the list of sets
done
555
0Wc7JGEjjUDM4vp2
Loop through the list of sets
done
Loop through the list of sets
done
556
AOS7fKldQL5DIcN7
Loop through the list of sets
done
Loop through the list of sets
done
557
ez2dRvG4hdbxv3r0
Loop through the list of sets
done
Loop through the list of sets
done
558
hwM4yvaymUCuH11P
Loop through the list of sets
done
Loop through the list of sets
done
559
cR5LYvKway60BZIk

Loop through the list of sets
done
Loop through the list of sets
done
640
nfK3euR4BJB8DPQE
Loop through the list of sets
done
Loop through the list of sets
done
641
Vfq19f8DOJuX1g9l
Loop through the list of sets
done
Loop through the list of sets
done
642
7rsf2fQnYdSADzO1
Loop through the list of sets
done
Loop through the list of sets
done
643
0Tw7xPCFY8Kkznev
Loop through the list of sets
done
Loop through the list of sets
done
644
HoiRjqhVT5ORnDlc
Loop through the list of sets
done
Loop through the list of sets
done
645
t61gRUmCesgyaG6K
Loop through the list of sets
done
Loop through the list of sets
done
646
yRkXMyJccHkf7j1B
Loop through the list of sets
done
Loop through the list of sets
done
647
qEwLVXUeIUlKkLwQ
Loop through the list of sets
done
Loop through the list of sets
done
648
8AvJcHLboetxZZf9
Loop through the list of sets
done
Loop through the list of sets
done
649
PWS8XcQ49VSJyvYq
Loop through the list of sets
done
Loop through the list of sets
done
650
BeoaY3fYbDaFmLjN

Loop through the list of sets
done
Loop through the list of sets
done
731
YxYYUuppP77qb0yh
Loop through the list of sets
done
Loop through the list of sets
done
732
C36GeIQoIdB1cEzT
Loop through the list of sets
done
Loop through the list of sets
done
733
wyNMFyeMs6kUlqNm
Loop through the list of sets
done
Loop through the list of sets
done
734
hF5Y5hbzwT9MkinU
Loop through the list of sets
done
Loop through the list of sets
done
735
KnkA0U2ytjyeYiME
Loop through the list of sets
done
Loop through the list of sets
done
736
TVH1r75HzcWGyt3m
Loop through the list of sets
done
Loop through the list of sets
done
737
Wpbfmpa4OH4SgoTH
Loop through the list of sets
done
Loop through the list of sets
done
738
eOyMnpLaNgaDBVgl
Loop through the list of sets
done
Loop through the list of sets
done
739
ZbcxnQ1aCzeNNG2l
Loop through the list of sets
done
Loop through the list of sets
done
740
JGxjtyllaWOPM2k7
Loop through the list of sets
done
Loop through the list of sets
done
741
TPyMvaeUS5Gugqn6

Loop through the list of sets
done
Loop through the list of sets
done
822
lp6dT5ZbHwGzzcEE
Loop through the list of sets
done
Loop through the list of sets
done
823
X1hvP89yKi3fqnkx
Loop through the list of sets
done
Loop through the list of sets
done
824
fTPE94ZUbrCGbGNf
Loop through the list of sets
done
Loop through the list of sets
done
825
MSsSZSiFyLNxaZO4
Loop through the list of sets
done
Loop through the list of sets
done
826
CMoYcyU4kMOa3sVY
Loop through the list of sets
done
Loop through the list of sets
done
827
jOEL85ImTTwtWGYP
Loop through the list of sets
done
Loop through the list of sets
done
828
ynOnBqKs9FjgYw7B
Loop through the list of sets
done
Loop through the list of sets
done
829
BC6GrX7aecdpkLqr
Loop through the list of sets
done
Loop through the list of sets
done
830
lzNMBHvI4MYaHzhg
Loop through the list of sets
done
Loop through the list of sets
done
831
VlyPBH0EpsKX3zWB
Loop through the list of sets
done
Loop through the list of sets
done
832
Urc71dvcvg5tcbYc

In [ ]:
array_embeddings = [tfidf,word2vec,bert, blf_correlation,ivec256, mfcc_stats, musicnn]
def get_avg_metric_at_10(array_embedding):
    return [get_avg_precision_recall_at_10(embedding) for embedding in array_embedding]
   

In [ ]:
#TODO: try this
'''import polars as pl

def calc_avg_recall_precision_at_10(df):
    # Define the value of k
    k = 10

    # Sort the dataframe by similarity scores in descending order
    ranked_df = df.sort('similarity_score_column', reverse=True)

    # Select the top k tracks from the ranked list
    retrieved_df = ranked_df.head(k)

    # Get the genres of the query tracks
    query_genres = genres.filter(df['id'].is_in(retrieved_df['id']))['genre']

    # Convert genres to sets
    query_genres = query_genres.apply(lambda genre_str: set(genre_str.split(',')))

    # Get the genres of the top k tracks
    top_k_genres = retrieved_df['genre'].apply(lambda genre_str: set(genre_str.split(',')))

    # Get count of all relevant tracks in the dataset
    all_relevant = query_genres.count()

    # Get count of all relevant tracks in top k
    k_relevant = top_k_genres.explode().filter(query_genres.explode()).count()

    # Calculate recall and precision
    recall = k_relevant / all_relevant
    precision = k_relevant / k

    # Print the results
    print(f'Average Precision@{k}: {precision:.9f}')
    print(f'Average Recall@{k}: {recall:.9f}')

    return precision, recall

'''

In [ ]:
get_avg_metric_at_10(array_embeddings)


In [ ]:
index_value

In [ ]:
df = pd.merge(df, genres, on='id')

In [ ]:
ranked_list = df.sort_values(by='zyzILCQvVeUFIINi', ascending=False).genre.tolist()

In [ ]:
ranked_list

In [ ]:
# Die list comprehension, die jeden String in der Spalte genre in ein Set umwandelt und zum Array hinzufügt
array = [set(df.genre[i].split(",")[0][2:], df.genre[i].split(",")[-1][:-2]) for i in range(len(df.genre))]

# Das Array aus Sets ausgeben
print(array)

In [ ]:
# Erstellen Sie ein leeres Array aus Sets
array = []

# Iterieren Sie über die Spalte in Ihrem DataFrame
for s in df['genre']:
  # Wandeln Sie den String in eine Liste von Genres um
  l = s.strip("[]").split(",")
  # Entfernen Sie die Anführungszeichen um jedes Genre
  l = [g.strip("'") for g in l]
  # Wandeln Sie die Liste in ein Set um
  s = set(l)
  # Fügen Sie das Set zum Array hinzu
  array.append(s)

# Das Array aus Sets ist fertig
print(array)

In [ ]:
l

In [ ]:
type(df.index[1])


In [ ]:
get_avg_recall_at_k(blf_correlation, 10, genres)

In [ ]:
type(genres["id"][1])

In [ ]:
genres[genres["id"]== "02RGE9FNH65RtMS7"]


In [ ]:
# Import numpy
import numpy as np

# Define a sample repr data set with 5 vectors, one of them having a zero norm
repr = blf_correlation
repr = repr.iloc[:, 2:].to_numpy()

# Calculate the norm of each vector along the rows
norms = np.linalg.norm(repr, axis=1)

# Create a boolean mask of the condition that the norm is zero
mask = np.isclose(norms, 0)

# Use numpy.nonzero to find the indices of the vectors that have a zero norm
indices = np.nonzero(mask)

# Use numpy indexing to get the problematic vectors
problematic_vectors = repr[indices]

# Print the indices and the problematic vectors
print("The indices of the problematic vectors are:", indices)
print("The problematic vectors are:", problematic_vectors)


In [ ]:
"""
Initialize empty dictionary to store the results

User guide:
the dictionary has three dimensions:
1st dim: query tracks 
available tags ["track1", "track2", "track3"]
2nd dim: retrieval systems 
available tags ["base_line", "tfidf", "word2vec", "bert", "blf_correlation", "ivec256", "mfcc_stats", "musicnn"]
3rd dim: evaluation metrics and retrieved tracks 
available tags ["tracks", "precision@10", "recall@10", "genre_diversity@10", "genre_coverage@10", "ndcg"]

Example:
res["track1"]["base_line"]["tracks"]

"""
with open('result_mod.json', 'r') as f:
    res = json.load(f)

## Track 1 
"Love Me" "The 1975"

In [ ]:
name = "Love Me"
artist = "The 1975"
id_track1 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track1"]["base_line"]["tracks"] = random_baseline(id=id_track1, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track1"]["tfidf"]["tracks"] = text_based(id=id_track1, repr=tfidf, N=10, sim_func=cos_sim)
res["track1"]["word2vec"]["tracks"] = text_based(id=id_track1, repr=word2vec, N=10, sim_func=cos_sim)
res["track1"]["bert"]["tracks"] = text_based(id=id_track1, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track1"]["blf_correlation"]["tracks"] = audio_based(id=id_track1, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track1"]["blf_correlation"]["tracks"], info)

res["track1"]["ivec256"]["tracks"] = audio_based(id=id_track1, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec256 Results:")
display_res(res["track1"]["ivec256"]["tracks"], info)

res["track1"]["mfcc_stats"]["tracks"] = audio_based(id=id_track1, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("MFCC Stats Results:")
display_res(res["track1"]["mfcc_stats"]["tracks"], info)

res["track1"]["musicnn"]["tracks"] = audio_based(id=id_track1, repr=musicnn, N=10, sim_func=cos_sim)
print("Musicnn Results:")
display_res(res["track1"]["musicnn"]["tracks"], info)

In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q1_genres_mfcc_stats = get_genre_from_ids(res["track1"]["mfcc_stats"]["tracks"],genres)
q1_genres_blf_correlation = get_genre_from_ids(res["track1"]["blf_correlation"]["tracks"], genres)
q1_genres_ivec256 = get_genre_from_ids(res["track1"]["ivec256"]["tracks"], genres)
q1_genres_musicnn = get_genre_from_ids(res["track1"]["musicnn"]["tracks"], genres)
q1_genres_tfidf = get_genre_from_ids(res["track1"]["tfidf"]["tracks"], genres)
q1_genres_word2vec = get_genre_from_ids(res["track1"]["word2vec"]["tracks"],genres)
q1_genres_bert =  get_genre_from_ids(res["track1"]["bert"]["tracks"],genres)
q1_genres_base_line = get_genre_from_ids(res["track1"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track1

#get query genre
query_genre1 = get_genre_from_query(id_track1, genres)

#query track 1 precision (k=10) for all 8 Retrieval Systems
res["track1"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_mfcc_stats,10)
precision = res["track1"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_blf_correlation,10)
precision = res["track1"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_ivec256,10)
precision = res["track1"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track1"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_musicnn,10)
precision = res["track1"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre1,q1_genres_tfidf,10)
precision = res["track1"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_word2vec,10)
precision = res["track1"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track1"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_bert,10)
precision = res["track1"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track1"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre1,q1_genres_base_line,10)
precision = res["track1"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")

In [ ]:
genres_list = genres.values.tolist()
res["track1"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k_vectorized(query_genre1, q1_genres_mfcc_stats,genres_list,10)
recall = res["track1"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
genres.values.tolist()


In [ ]:
#query track 1 recall (k=10) for all 8 Retrieval Systems
#create genres dataset list for later use


res["track1"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_mfcc_stats,genres_list,10)
recall = res["track1"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_blf_correlation,genres_list,10)
recall = res["track1"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_ivec256,genres_list,10)
recall = res["track1"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_musicnn,genres_list,10)
recall = res["track1"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_tfidf,genres_list,10)
recall = res["track1"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre1, q1_genres_word2vec,genres_list,10)
recall = res["track1"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre1,  q1_genres_bert,genres_list,10)
recall = res["track1"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track1"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre1, q1_genres_base_line,genres_list,10)
recall = res["track1"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q1_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track1, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q1_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track1, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q1_100_ivec256 = get_genre_from_ids(audio_based(id=id_track1, repr=ivec256, N=100, sim_func=cos_sim), genres)
q1_100_musicnn = get_genre_from_ids(audio_based(id=id_track1, repr=musicnn, N=100, sim_func=cos_sim), genres)
#retrieve 100 text based results for precision recall plot because k should be varied in the interval [1,100]
q1_100_tfidf = get_genre_from_ids(text_based(id=id_track1, repr=tfidf, N=100, sim_func=cos_sim), genres)
q1_100_word2vec = get_genre_from_ids(text_based(id=id_track1, repr=word2vec, N=100, sim_func=cos_sim),genres)
q1_100_bert =  get_genre_from_ids(text_based(id=id_track1, repr=bert, N=100, sim_func=cos_sim),genres)
q1_100_base_line = get_genre_from_ids(random_baseline(id=id_track1, info=info, N=100),genres)


#plot precision recall curve for query track1 for all evaluated systems

system_data1 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre1, "retrieved_genres": q1_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre1, "retrieved_genres": q1_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre1, "retrieved_genres": q1_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre1, "retrieved_genres": q1_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre1, "retrieved_genres": q1_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre1, "retrieved_genres": q1_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre1, "retrieved_genres": q1_genres_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre1, "retrieved_genres": q1_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data1)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q1_genre_mfcc_stats = [get_genre(id, genres) for id in res["track1"]["mfcc_stats"]["tracks"]]
q1_genre_blf_correlation = [get_genre(id, genres) for id in res["track1"]["blf_correlation"]["tracks"]]
q1_genre_ivec256 = [get_genre(id, genres) for id in res["track1"]["ivec256"]["tracks"]]
q1_genre_musicnn = [get_genre(id, genres) for id in res["track1"]["musicnn"]["tracks"]]
#text based
q1_genre_tfidf = [get_genre(id, genres) for id in res["track1"]["tfidf"]["tracks"]]
q1_genre_word2vec = [get_genre(id, genres) for id in res["track1"]["word2vec"]["tracks"]]
q1_genre_bert = [get_genre(id, genres) for id in res["track1"]["bert"]["tracks"]]
q1_genre_base_line = [get_genre(id, genres) for id in res["track1"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track1"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q1_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track1"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q1_genre_blf_correlation,all_genres,N=10)
genre_div = res["track1"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["ivec256"]["genre_diversity@10"] = gen_div_10(q1_genre_ivec256,all_genres,N=10)
genre_div = res["track1"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["musicnn"]["genre_diversity@10"] = gen_div_10(q1_genre_musicnn,all_genres,N=10)
genre_div = res["track1"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["tfidf"]["genre_diversity@10"] = gen_div_10(q1_genre_tfidf,all_genres,N=10)
genre_div = res["track1"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["word2vec"]["genre_diversity@10"] = gen_div_10(q1_genre_word2vec,all_genres,N=10)
genre_div = res["track1"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["bert"]["genre_diversity@10"] = gen_div_10(q1_genre_bert,all_genres,N=10)
genre_div = res["track1"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track1"]["base_line"]["genre_diversity@10"] = gen_div_10(q1_genre_base_line,all_genres,N=10)
genre_div = res["track1"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
print(query_genre1)

In [ ]:
# calculate genre coverage@10 

res["track1"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track1"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track1"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track1"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track1"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track1"]["ivec256"]["tracks"], genres)
genre_cov = res["track1"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track1"]["musicnn"]["tracks"], genres)
genre_cov = res["track1"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track1"]["tfidf"]["tracks"], genres)
genre_cov = res["track1"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track1"]["word2vec"]["tracks"], genres)
genre_cov = res["track1"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track1"]["bert"]["tracks"], genres)
genre_cov = res["track1"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track1"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track1"]["base_line"]["tracks"], genres)
genre_cov = res["track1"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 

res["track1"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track1, res["track1"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track1"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["blf_correlation"]["ndcg"] = ndcg_score(id_track1, res["track1"]["blf_correlation"]["tracks"], genres)
ndcg = res["track1"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["ivec256"]["ndcg"] = ndcg_score(id_track1, res["track1"]["ivec256"]["tracks"], genres)
ndcg = res["track1"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["musicnn"]["ndcg"] = ndcg_score(id_track1, res["track1"]["musicnn"]["tracks"], genres)
ndcg = res["track1"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["tfidf"]["ndcg"] = ndcg_score(id_track1, res["track1"]["tfidf"]["tracks"], genres)
ndcg = res["track1"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["word2vec"]["ndcg"] = ndcg_score(id_track1, res["track1"]["word2vec"]["tracks"], genres)
ndcg = res["track1"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["bert"]["ndcg"] = ndcg_score(id_track1, res["track1"]["bert"]["tracks"], genres)
ndcg = res["track1"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track1"]["base_line"]["ndcg"] = ndcg_score(id_track1, res["track1"]["base_line"]["tracks"], genres)
ndcg = res["track1"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")

## Track 2 
"One" "U2"

In [ ]:
name = 'One'
artist = 'U2'
id_track2 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track2"]["base_line"]["tracks"] = random_baseline(id=id_track2, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track2"]["tfidf"]["tracks"] = text_based(id=id_track2, repr=tfidf, N=10, sim_func=cos_sim)
res["track2"]["word2vec"]["tracks"] = text_based(id=id_track2, repr=word2vec, N=10, sim_func=cos_sim)
res["track2"]["bert"]["tracks"] = text_based(id=id_track2, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track2"]["blf_correlation"]["tracks"] = audio_based(id=id_track2, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track2"]["blf_correlation"]["tracks"], info)

res["track2"]["ivec256"]["tracks"] = audio_based(id=id_track2, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec 256 Results:")
display_res(res["track2"]["ivec256"]["tracks"], info)

res["track2"]["mfcc_stats"]["tracks"] = audio_based(id=id_track2, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("mfcc_stats Results:")
display_res(res["track2"]["mfcc_stats"]["tracks"], info)

res["track2"]["musicnn"]["tracks"] = audio_based(id=id_track2, repr=musicnn, N=10, sim_func=cos_sim)
print("musicnn Results:")
display_res(res["track2"]["musicnn"]["tracks"], info)

In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q2_genres_mfcc_stats = get_genre_from_ids(res["track2"]["mfcc_stats"]["tracks"],genres)
q2_genres_blf_correlation = get_genre_from_ids(res["track2"]["blf_correlation"]["tracks"], genres)
q2_genres_ivec256 = get_genre_from_ids(res["track2"]["ivec256"]["tracks"], genres)
q2_genres_musicnn = get_genre_from_ids(res["track2"]["musicnn"]["tracks"], genres)
q2_genres_tfidf = get_genre_from_ids(res["track2"]["tfidf"]["tracks"], genres)
q2_genres_word2vec = get_genre_from_ids(res["track2"]["word2vec"]["tracks"],genres)
q2_genres_bert =  get_genre_from_ids(res["track2"]["bert"]["tracks"],genres)
q2_genres_base_line = get_genre_from_ids(res["track2"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track2
#get query genre
query_genre2 = get_genre_from_query(id_track2, genres)

#query track 1 precision (k=10) for all 8 Retrieval Systems
res["track2"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_mfcc_stats,10)
precision = res["track2"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_blf_correlation,10)
precision = res["track2"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_ivec256,10)
precision = res["track2"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track2"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_musicnn,10)
precision = res["track2"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre2,q2_genres_tfidf,10)
precision = res["track2"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_word2vec,10)
precision = res["track2"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track2"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_bert,10)
precision = res["track2"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track2"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre2,q2_genres_base_line,10)
precision = res["track2"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")


In [ ]:
#query track 2 recall (k=10) for all 8 Retrieval Systems
#create genres dataset list for later use
genres_list = genres.values.tolist()

res["track2"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_mfcc_stats,genres_list,10)
recall = res["track2"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_blf_correlation,genres_list,10)
recall = res["track2"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_ivec256,genres_list,10)
recall = res["track2"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre2, q2_genres_musicnn,genres_list,10)
recall = res["track2"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre2, q2_genres_tfidf,genres_list,10)
recall = res["track2"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre2, q2_genres_word2vec,genres_list,10)
recall = res["track2"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre2,q2_genres_bert,genres_list,10)
recall = res["track2"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track2"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre2,q2_genres_base_line,genres_list,10)
recall = res["track2"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q2_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track2, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q2_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track2, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q2_100_ivec256 = get_genre_from_ids(audio_based(id=id_track2, repr=ivec256, N=100, sim_func=cos_sim), genres)
q2_100_musicnn = get_genre_from_ids(audio_based(id=id_track2, repr=musicnn, N=100, sim_func=cos_sim), genres)
q2_100_tfidf = get_genre_from_ids(text_based(id=id_track2, repr=tfidf, N=100, sim_func=cos_sim), genres)
q2_100_word2vec = get_genre_from_ids(text_based(id=id_track2, repr=word2vec, N=100, sim_func=cos_sim),genres)
q2_100_bert =  get_genre_from_ids(text_based(id=id_track2, repr=bert, N=100, sim_func=cos_sim),genres)
q2_100_base_line = get_genre_from_ids(random_baseline(id=id_track2, info=info, N=100),genres)

#plot precision recall curve for query track2 for all evaluated systems

system_data2 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre2, "retrieved_genres": q2_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre2, "retrieved_genres": q2_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre2, "retrieved_genres": q2_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre2, "retrieved_genres": q2_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre2, "retrieved_genres": q2_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre2, "retrieved_genres": q2_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre2, "retrieved_genres": q2_genres_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre2, "retrieved_genres": q2_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data2)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q2_genre_mfcc_stats = [get_genre(id, genres) for id in res["track2"]["mfcc_stats"]["tracks"]]
q2_genre_blf_correlation = [get_genre(id, genres) for id in res["track2"]["blf_correlation"]["tracks"]]
q2_genre_ivec256 = [get_genre(id, genres) for id in res["track2"]["ivec256"]["tracks"]]
q2_genre_musicnn = [get_genre(id, genres) for id in res["track2"]["musicnn"]["tracks"]]
#text based
q2_genre_tfidf = [get_genre(id, genres) for id in res["track2"]["tfidf"]["tracks"]]
q2_genre_word2vec = [get_genre(id, genres) for id in res["track2"]["word2vec"]["tracks"]]
q2_genre_bert = [get_genre(id, genres) for id in res["track2"]["bert"]["tracks"]]
q2_genre_random = [get_genre(id, genres) for id in res["track2"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track2"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q2_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track2"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q2_genre_blf_correlation,all_genres,N=10)
genre_div = res["track2"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["ivec256"]["genre_diversity@10"] = gen_div_10(q2_genre_ivec256,all_genres,N=10)
genre_div = res["track2"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["musicnn"]["genre_diversity@10"] = gen_div_10(q2_genre_musicnn,all_genres,N=10)
genre_div = res["track2"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["tfidf"]["genre_diversity@10"] = gen_div_10(q2_genre_tfidf,all_genres,N=10)
genre_div = res["track2"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["word2vec"]["genre_diversity@10"] = gen_div_10(q2_genre_word2vec,all_genres,N=10)
genre_div = res["track2"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["bert"]["genre_diversity@10"] = gen_div_10(q2_genre_bert,all_genres,N=10)
genre_div = res["track2"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track2"]["base_line"]["genre_diversity@10"] = gen_div_10(q2_genre_random,all_genres,N=10)
genre_div = res["track2"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
# calculate genre coverage@10 
res["track2"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track2"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track2"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track2"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track2"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track2"]["ivec256"]["tracks"], genres)
genre_cov = res["track2"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track2"]["musicnn"]["tracks"], genres)
genre_cov = res["track2"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track2"]["tfidf"]["tracks"], genres)
genre_cov = res["track2"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track2"]["word2vec"]["tracks"], genres)
genre_cov = res["track2"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track2"]["bert"]["tracks"], genres)
genre_cov = res["track2"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track2"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track2"]["base_line"]["tracks"], genres)
genre_cov = res["track2"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 
res["track2"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track2, res["track2"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track2"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["blf_correlation"]["ndcg"] = ndcg_score(id_track2, res["track2"]["blf_correlation"]["tracks"], genres)
ndcg = res["track2"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["ivec256"]["ndcg"] = ndcg_score(id_track2, res["track2"]["ivec256"]["tracks"], genres)
ndcg = res["track2"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["musicnn"]["ndcg"] = ndcg_score(id_track2, res["track2"]["musicnn"]["tracks"], genres)
ndcg = res["track2"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["tfidf"]["ndcg"] = ndcg_score(id_track2, res["track2"]["tfidf"]["tracks"], genres)
ndcg = res["track2"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["word2vec"]["ndcg"] = ndcg_score(id_track2, res["track2"]["word2vec"]["tracks"], genres)
ndcg = res["track2"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["bert"]["ndcg"] = ndcg_score(id_track2, res["track2"]["bert"]["tracks"], genres)
ndcg = res["track2"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track2"]["base_line"]["ndcg"] = ndcg_score(id_track2, res["track2"]["base_line"]["tracks"], genres)
ndcg = res["track2"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")


## Track 3
"Every Christmas" "Kelly Clarkson"

In [ ]:
name = 'Every Christmas'
artist = 'Kelly Clarkson'
id_track3 = get_id_from_info(name, artist, info)

In [ ]:
# retrieve tracks with random baseline 
res["track3"]["base_line"]["tracks"] = random_baseline(id=id_track3, info=info, N=10)

In [ ]:
# retrieve tracks with text based retrieval systems 
res["track3"]["tfidf"]["tracks"] = text_based(id=id_track3, repr=tfidf, N=10, sim_func=cos_sim)
res["track3"]["word2vec"]["tracks"] = text_based(id=id_track3, repr=word2vec, N=10, sim_func=cos_sim)
res["track3"]["bert"]["tracks"] = text_based(id=id_track3, repr=bert, N=10, sim_func=cos_sim)

In [ ]:
# retrieve tracks with audio based retrieval systems 
res["track3"]["blf_correlation"]["tracks"] = audio_based(id=id_track3, repr=blf_correlation, N=10, sim_func=cos_sim)
print("blf correlation Results:")
display_res(res["track3"]["blf_correlation"]["tracks"], info)

res["track3"]["ivec256"]["tracks"] = audio_based(id=id_track3, repr=ivec256, N=10, sim_func=cos_sim)
print("ivec256 Results:")
display_res(res["track3"]["ivec256"]["tracks"], info)

res["track3"]["mfcc_stats"]["tracks"] = audio_based(id=id_track3, repr=mfcc_stats, N=10, sim_func=cos_sim)
print("mfcc_stats Results:")
display_res(res["track3"]["mfcc_stats"]["tracks"], info)

res["track3"]["musicnn"]["tracks"] = audio_based(id=id_track3, repr=musicnn, N=10, sim_func=cos_sim)
print("musicnn Results:")
display_res(res["track3"]["musicnn"]["tracks"], info)


In [ ]:
#retrieve genres of the results for precision@10 & recall@10 calculation
q3_genres_mfcc_stats = get_genre_from_ids(res["track3"]["mfcc_stats"]["tracks"],genres)
q3_genres_blf_correlation = get_genre_from_ids(res["track3"]["blf_correlation"]["tracks"], genres)
q3_genres_ivec256 = get_genre_from_ids(res["track3"]["ivec256"]["tracks"], genres)
q3_genres_musicnn = get_genre_from_ids(res["track3"]["musicnn"]["tracks"], genres)
q3_genres_tfidf = get_genre_from_ids(res["track3"]["tfidf"]["tracks"], genres)
q3_genres_word2vec = get_genre_from_ids(res["track3"]["word2vec"]["tracks"],genres)
q3_genres_bert =  get_genre_from_ids(res["track3"]["bert"]["tracks"],genres)
q3_genres_base_line = get_genre_from_ids(res["track3"]["base_line"]["tracks"],genres)

In [ ]:
# calculate precision @10 for track3

#get query genre
query_genre3 = get_genre_from_query(id_track3, genres)

res["track3"]["mfcc_stats"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_mfcc_stats,10)
precision = res["track3"]["mfcc_stats"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["blf_correlation"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_blf_correlation,10)
precision = res["track3"]["blf_correlation"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["ivec256"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_ivec256,10)
precision = res["track3"]["ivec256"]["precision@10"] 
print(f"precision@10: {precision}")

res["track3"]["musicnn"]["precision@10"] = calculate_precision_at_k(query_genre3,q2_genres_musicnn,10)
precision = res["track3"]["musicnn"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["tfidf"]["precision@10"]= calculate_precision_at_k(query_genre3,q3_genres_tfidf,10)
precision = res["track3"]["tfidf"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["word2vec"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_word2vec,10)
precision = res["track3"]["word2vec"]["precision@10"]
print(f"precision@10: {precision}")

res["track3"]["bert"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_bert,10)
precision = res["track3"]["bert"]["precision@10"] 
print(f"precision@10: {precision}")

res["track3"]["base_line"]["precision@10"] = calculate_precision_at_k(query_genre3,q3_genres_base_line,10)
precision = res["track3"]["base_line"]["precision@10"]
print(f"precision@10: {precision}")

In [ ]:
#query track 3 recall (k=10) for all 8 Retrieval Systems

res["track3"]["mfcc_stats"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_mfcc_stats,genres_list,10)
recall = res["track3"]["mfcc_stats"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["blf_correlation"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_blf_correlation,genres_list,10)
recall = res["track3"]["blf_correlation"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["ivec256"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_ivec256,genres_list,10)
recall = res["track3"]["ivec256"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["musicnn"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_musicnn,genres_list,10)
recall = res["track3"]["musicnn"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["tfidf"]["recall@10"] = calculate_recall_at_k(query_genre3, q3_genres_tfidf,genres_list,10)
recall = res["track3"]["tfidf"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["word2vec"]["recall@10"]= calculate_recall_at_k(query_genre3, q3_genres_word2vec,genres_list,10)
recall = res["track3"]["word2vec"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["bert"]["recall@10"]= calculate_recall_at_k(query_genre3,q3_genres_bert,genres_list,10)
recall = res["track3"]["bert"]["recall@10"]
print(f"recall@10: {recall}")

res["track3"]["base_line"]["recall@10"] = calculate_recall_at_k(query_genre3,q3_genres_base_line,genres_list,10)
recall = res["track3"]["base_line"]["recall@10"]
print(f"recall@10: {recall}")

In [ ]:
#retrieve 100 results for precision-recall plot (audio based) because k should be varied in the interval [1,100]
q3_100_mfcc_stats = get_genre_from_ids(audio_based(id=id_track3, repr=mfcc_stats, N=100, sim_func=cos_sim), genres)
q3_100_blf_correlation = get_genre_from_ids(audio_based(id=id_track3, repr=blf_correlation, N=100, sim_func=cos_sim), genres)
q3_100_ivec256 = get_genre_from_ids(audio_based(id=id_track3, repr=ivec256, N=100, sim_func=cos_sim), genres)
q3_100_musicnn = get_genre_from_ids(audio_based(id=id_track3, repr=musicnn, N=100, sim_func=cos_sim), genres)
q3_100_tfidf = get_genre_from_ids(text_based(id=id_track3, repr=tfidf, N=100, sim_func=cos_sim), genres)
q3_100_word2vec = get_genre_from_ids(text_based(id=id_track3, repr=word2vec, N=100, sim_func=cos_sim),genres)
q3_100_bert =  get_genre_from_ids(text_based(id=id_track3, repr=bert, N=100, sim_func=cos_sim),genres)
q3_100_base_line = get_genre_from_ids(random_baseline(id=id_track3, info=info, N=100),genres)

#plot precision recall curve for query track3 for all evaluated systems

system_data3 = {
    "audio mfcc stats": {"system_name": "Audio MFCC Stats","query_genre": query_genre3, "retrieved_genres": q3_100_mfcc_stats, "dataset_genres": genres_list},
    "audio blf correlation": {"system_name": "Audio BLF Correlation","query_genre": query_genre3, "retrieved_genres": q3_100_blf_correlation, "dataset_genres": genres_list},
    "audio ivec_256": {"system_name": "Audio iVec 256","query_genre": query_genre3, "retrieved_genres": q3_100_ivec256, "dataset_genres":genres_list},
    "audio muiscnn": {"system_name": "Audio Musicnn","query_genre": query_genre3, "retrieved_genres": q3_100_musicnn, "dataset_genres":genres_list},
    "text tf-idf": {"system_name": "Text TF-IDF","query_genre": query_genre3, "retrieved_genres": q3_100_tfidf, "dataset_genres":genres_list},
    "text word2vec": {"system_name": "Text Word2Vec","query_genre": query_genre3, "retrieved_genres": q3_100_word2vec, "dataset_genres":genres_list},
    "text bert": {"system_name": "Text BERT","query_genre": query_genre3, "retrieved_genres": q3_100_bert, "dataset_genres":genres_list},
    "text random": {"system_name": "Text Random Baseline","query_genre": query_genre3, "retrieved_genres": q3_100_base_line, "dataset_genres":genres_list},
}

plot_precision_recall_curve(system_data3)

In [ ]:
# calculate genre diversity@10
all_genres = list(set([item for id in genres['id'] for item in get_genre(id, genres)]))

#audio based
q3_genre_mfcc_stats = [get_genre(id, genres) for id in res["track3"]["mfcc_stats"]["tracks"]]
q3_genre_blf_correlation = [get_genre(id, genres) for id in res["track3"]["blf_correlation"]["tracks"]]
q3_genre_ivec256 = [get_genre(id, genres) for id in res["track3"]["ivec256"]["tracks"]]
q3_genre_musicnn = [get_genre(id, genres) for id in res["track3"]["musicnn"]["tracks"]]
#text based
q3_genre_tfidf = [get_genre(id, genres) for id in res["track3"]["tfidf"]["tracks"]]
q3_genre_word2vec = [get_genre(id, genres) for id in res["track3"]["word2vec"]["tracks"]]
q3_genre_bert = [get_genre(id, genres) for id in res["track3"]["bert"]["tracks"]]
q3_genre_random = [get_genre(id, genres) for id in res["track3"]["base_line"]["tracks"]]

#calculate genre diversity @10
res["track3"]["mfcc_stats"]["genre_diversity@10"] = gen_div_10(q3_genre_mfcc_stats, all_genres, N=10)
genre_div = res["track3"]["mfcc_stats"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["blf_correlation"]["genre_diversity@10"] = gen_div_10(q3_genre_blf_correlation,all_genres,N=10)
genre_div = res["track3"]["blf_correlation"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["ivec256"]["genre_diversity@10"] = gen_div_10(q3_genre_ivec256,all_genres,N=10)
genre_div = res["track3"]["ivec256"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["musicnn"]["genre_diversity@10"] = gen_div_10(q3_genre_musicnn,all_genres,N=10)
genre_div = res["track3"]["musicnn"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["tfidf"]["genre_diversity@10"] = gen_div_10(q3_genre_tfidf,all_genres,N=10)
genre_div = res["track3"]["tfidf"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["word2vec"]["genre_diversity@10"] = gen_div_10(q3_genre_word2vec,all_genres,N=10)
genre_div = res["track3"]["word2vec"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["bert"]["genre_diversity@10"] = gen_div_10(q3_genre_bert,all_genres,N=10)
genre_div = res["track3"]["bert"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")

res["track3"]["base_line"]["genre_diversity@10"] = gen_div_10(q3_genre_random,all_genres,N=10)
genre_div = res["track3"]["base_line"]["genre_diversity@10"]
print(f"genre diversity@10: {genre_div}")


In [ ]:
# calculate genre coverage@10 
res["track3"]["mfcc_stats"]["genre_coverage@10"] = gen_cov_10(res["track3"]["mfcc_stats"]["tracks"], genres)
genre_cov = res["track3"]["mfcc_stats"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["blf_correlation"]["genre_coverage@10"] = gen_cov_10(res["track3"]["blf_correlation"]["tracks"], genres)
genre_cov = res["track3"]["blf_correlation"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["ivec256"]["genre_coverage@10"] = gen_cov_10(res["track3"]["ivec256"]["tracks"], genres)
genre_cov = res["track3"]["ivec256"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["musicnn"]["genre_coverage@10"] = gen_cov_10(res["track3"]["musicnn"]["tracks"], genres)
genre_cov = res["track3"]["musicnn"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["tfidf"]["genre_coverage@10"] = gen_cov_10(res["track3"]["tfidf"]["tracks"], genres)
genre_cov = res["track3"]["tfidf"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["word2vec"]["genre_coverage@10"] = gen_cov_10(res["track3"]["word2vec"]["tracks"], genres)
genre_cov = res["track3"]["word2vec"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["bert"]["genre_coverage@10"] = gen_cov_10(res["track3"]["bert"]["tracks"], genres)
genre_cov = res["track3"]["bert"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

res["track3"]["base_line"]["genre_coverage@10"] = gen_cov_10(res["track3"]["base_line"]["tracks"], genres)
genre_cov = res["track3"]["base_line"]["genre_coverage@10"]
print(f"genre coverage@10: {genre_cov}")

In [ ]:
# calculate ndcg@10 
res["track3"]["mfcc_stats"]["ndcg"] = ndcg_score(id_track3, res["track3"]["mfcc_stats"]["tracks"], genres)
ndcg = res["track3"]["mfcc_stats"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["blf_correlation"]["ndcg"] = ndcg_score(id_track3, res["track3"]["blf_correlation"]["tracks"], genres)
ndcg = res["track3"]["blf_correlation"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["ivec256"]["ndcg"] = ndcg_score(id_track3, res["track3"]["ivec256"]["tracks"], genres)
ndcg = res["track3"]["ivec256"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["musicnn"]["ndcg"] = ndcg_score(id_track3, res["track3"]["musicnn"]["tracks"], genres)
ndcg = res["track3"]["musicnn"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["tfidf"]["ndcg"] = ndcg_score(id_track3, res["track3"]["tfidf"]["tracks"], genres)
ndcg = res["track3"]["tfidf"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["word2vec"]["ndcg"] = ndcg_score(id_track3, res["track3"]["word2vec"]["tracks"], genres)
ndcg = res["track3"]["word2vec"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["bert"]["ndcg"] = ndcg_score(id_track3, res["track3"]["bert"]["tracks"], genres)
ndcg = res["track3"]["bert"]["ndcg"]
print(f"ndcg: {ndcg}")

res["track3"]["base_line"]["ndcg"] = ndcg_score(id_track3, res["track3"]["base_line"]["tracks"], genres)
ndcg = res["track3"]["base_line"]["ndcg"]
print(f"ndcg: {ndcg}")

In [ ]:
# Apply the vectorized function with k = 2
df["recall_at_2"] = df.apply(lambda row: calculate_recall_at_k_vectorized([row["query_genre"]], [row["retrieved_genres"]], [row["dataset_genres"]], 10), axis=1)

In [ ]:
blf_correlation

In [ ]:
# Importieren Sie sklearn
from sklearn.metrics.pairwise import cosine_similarity
# Berechnen Sie die Kosinus-Ähnlichkeit aller IDs mit allen anderen IDs entlang der Reihen
similarity = cosine_similarity(blf_correlation[:][2:])

# Erhalten Sie nur die obere Dreiecksmatrix der Ähnlichkeitsmatrix
upper = np.triu(similarity)

In [ ]:
df


In [ ]:
df_merged = df.merge( genres,left_index=True, right_on='id',  how='inner')

In [ ]:
df_merged

In [ ]:
genres

In [ ]:
blf_correlation_matrix

In [ ]:
 # Fill the lower triangular part with the same values as the upper triangular part
cos_sim_matrix = cos_sim_matrix + cos_sim_matrix.T - np.diag(cos_sim_matrix.diagonal())

In [ ]:
blf_correlation.shape